# Paillier Cryptosystem


The Paillier cryptosystem is a partial homomorphic encryption scheme which allows two types of computation:
- addition of two ciphertexts
- multiplication of a ciphertext by a plaintext number


In [1]:
import random

import galois

order = 2**8


# Helper functions


def rand_int(n=order - 1):
    return random.randint(0, n)


def rand_prime(n=order):
    return galois.random_prime(n)


# Greatest common divisor
def gcd(a, b):
    while b != 0:
        a, b = b, a % b
    return a


# Least common multiple
def lcm(a, b):
    return a * b // gcd(a, b)


# Modular multiplicative inverse
def modinv(a, m):
    m0, x0, x1 = m, 0, 1
    while a > 1:
        q = a // m
        m, a = a % m, m
        x0, x1 = x1 - q * x0, x0
    return x1 + m0 if x1 < 0 else x1


def L(x, n):
    return (x - 1) // n


def key_gen():
    # First, we need to find p and q
    while True:
        p = rand_prime()
        q = rand_prime()
        while gcd(p * q, (p - 1) * (q - 1)) != 1:
            p = rand_prime()
            q = rand_prime()

        n = p * q
        lam = lcm(p - 1, q - 1)
        g = rand_int(n**2)

        try:
            L_value = L(pow(g, lam, n**2), n)
            mu = modinv(L_value, n)
        except ZeroDivisionError:
            mu = None
        if mu is None:
            continue

        return (n, g), lam


def encrypt(n, g, m):
    assert m < n
    r = rand_int(n)
    c = pow(g, m, n**2) * pow(r, n, n**2) % n**2
    return c


def decrypt(lam, c, n, g):
    assert c < n**2

    # We are recalculating mu from known parameters
    L_value = L(pow(g, lam, n**2), n)
    mu = modinv(L_value, n)

    m = L(pow(c, lam, n**2), n) * mu % n
    return m


pk, sk = key_gen()

message = rand_int()
print(f"Original message: {message}")

encrypted = encrypt(*pk, message)
print(f"Encrypted message: {encrypted}")

decrypted = decrypt(sk, encrypted, *pk)
print(f"Decrypted message: {decrypted}")

assert message == decrypted, "Decryption failed!"

Original message: 87
Encrypted message: 274501753023609521124997256253246571386568943951164494136484192458069962359677540400812003499475392945392946686497498106133271006899564735192785122640447554055308291936331640389001866848253019590799405283526112213145868910100487986229134068038740779786072620657562439987326674247176461210254771972800642382417
Decrypted message: 87
